# Quick SAMPIC Waveform Viewer

Simple notebook for quick waveform visualization.

In [ ]:
import uproot
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (14, 6)

In [ ]:
# Load data
file = uproot.open('../output.root')
tree = file['events']

# Read first 50 events
events = tree.arrays(['sampic_event.hits.channel',
                      'sampic_event.hits.corrected_waveform',
                      'sampic_event.hits.amplitude',
                      'sampic_event.hits.baseline',
                      'sampic_event.hits.peak',
                      'sampic_event.hits.time_instant',
                      'sampic_event.hits.tot_value'], 
                     library='ak', entry_stop=50)

print(f"Loaded {len(events)} events")
print(f"Total entries in file: {tree.num_entries}")

In [ ]:
# Interactive waveform plotter
def plot_hit(event_num=0, hit_num=0):
    """Plot a single hit waveform."""
    if event_num >= len(events):
        print(f"Event {event_num} out of range (max: {len(events)-1})")
        return
    
    event = events[event_num]
    if hit_num >= len(event.sampic_event.hits):
        print(f"Hit {hit_num} out of range (event {event_num} has {len(event.sampic_event.hits)} hits)")
        return
    
    hit = event.sampic_event.hits[hit_num]
    waveform = np.array(hit.corrected_waveform)
    
    if len(waveform) == 0:
        print(f"Hit {hit_num} in event {event_num} has no waveform data")
        return
    
    fig, ax = plt.subplots(figsize=(14, 6))
    
    samples = np.arange(len(waveform))
    ax.plot(samples, waveform, 'b-', linewidth=2, label='Waveform')
    ax.axhline(y=hit.baseline, color='green', linestyle='--', linewidth=1.5, 
               label=f'Baseline: {hit.baseline:.1f} ADC')
    ax.axhline(y=hit.peak, color='red', linestyle='--', linewidth=1.5,
               label=f'Peak: {hit.peak:.1f} ADC')
    
    # Add text box with hit info
    info_text = f"""Event: {event_num}, Hit: {hit_num}
Channel: {hit.channel}
Amplitude: {hit.amplitude:.2f} ADC
TOT: {hit.tot_value:.2f} ns
Time: {hit.time_instant:.2f} ns
Samples: {len(waveform)}"""
    
    ax.text(0.02, 0.98, info_text, transform=ax.transAxes,
            verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5),
            fontfamily='monospace', fontsize=10)
    
    ax.set_xlabel('Sample Index', fontsize=12)
    ax.set_ylabel('Amplitude [ADC]', fontsize=12)
    ax.set_title(f'SAMPIC Waveform - Event {event_num}, Hit {hit_num}, Channel {hit.channel}', 
                 fontsize=14, fontweight='bold')
    ax.legend(loc='upper right', fontsize=10)
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Example: plot first hit from first event
plot_hit(0, 0)

In [ ]:
# Plot different hits - change these numbers!
plot_hit(event_num=1, hit_num=0)

In [ ]:
# Explore: find events with large amplitudes
import awkward as ak

for i, event in enumerate(events[:20]):
    for j, hit in enumerate(event.sampic_event.hits):
        if hit.amplitude > 100:  # Adjust threshold as needed
            print(f"Event {i}, Hit {j}: Ch {hit.channel}, Amp {hit.amplitude:.1f}, "
                  f"Samples {len(hit.corrected_waveform)}")

In [ ]:
# Plot all hits from one event
def plot_event(event_num=0):
    """Plot all hits from a single event."""
    if event_num >= len(events):
        print(f"Event {event_num} out of range")
        return
    
    event = events[event_num]
    num_hits = len(event.sampic_event.hits)
    
    if num_hits == 0:
        print(f"Event {event_num} has no hits")
        return
    
    fig, axes = plt.subplots(num_hits, 1, figsize=(14, 3*num_hits), squeeze=False)
    axes = axes.flatten()
    
    for i, hit in enumerate(event.sampic_event.hits):
        wf = np.array(hit.corrected_waveform)
        if len(wf) > 0:
            axes[i].plot(wf, 'b-', linewidth=1.5)
            axes[i].axhline(y=hit.baseline, color='g', linestyle='--', alpha=0.7)
            axes[i].axhline(y=hit.peak, color='r', linestyle='--', alpha=0.7)
            axes[i].set_ylabel('ADC')
            axes[i].set_title(f'Hit {i} | Ch {hit.channel} | Amp {hit.amplitude:.1f} | '
                             f'{len(wf)} samples', fontsize=10)
            axes[i].grid(True, alpha=0.3)
    
    axes[-1].set_xlabel('Sample Index')
    fig.suptitle(f'Event {event_num} - All {num_hits} Hits', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

# Plot first event
plot_event(0)

In [ ]:
# Quick statistics
all_amps = []
all_tots = []
all_channels = []

for event in events:
    for hit in event.sampic_event.hits:
        all_amps.append(hit.amplitude)
        all_tots.append(hit.tot_value)
        all_channels.append(hit.channel)

all_amps = np.array(all_amps)
all_tots = np.array(all_tots)
all_channels = np.array(all_channels)

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].hist(all_amps, bins=50, alpha=0.7, edgecolor='black')
axes[0].set_xlabel('Amplitude [ADC]')
axes[0].set_ylabel('Counts')
axes[0].set_title('Amplitude Distribution')
axes[0].set_yscale('log')

axes[1].hist(all_tots, bins=50, alpha=0.7, edgecolor='black', color='orange')
axes[1].set_xlabel('TOT [ns]')
axes[1].set_ylabel('Counts')
axes[1].set_title('Time over Threshold')

axes[2].hist(all_channels, bins=64, range=(0, 64), alpha=0.7, edgecolor='black', color='green')
axes[2].set_xlabel('Channel')
axes[2].set_ylabel('Hits')
axes[2].set_title('Channel Occupancy')

plt.tight_layout()
plt.show()

print(f"\nTotal hits: {len(all_amps)}")
print(f"Amplitude range: [{all_amps.min():.1f}, {all_amps.max():.1f}] ADC")
print(f"Mean amplitude: {all_amps.mean():.1f} ± {all_amps.std():.1f} ADC")
print(f"Active channels: {np.unique(all_channels)}")